In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn
plt.style.use("ggplot")

In [2]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

# Preprocessing

In [3]:
data_raw = pd.read_csv("monthly.csv")
y_raw = pd.read_csv("y.csv")

In [17]:
y_raw.head()

,date,TICKER,Y,dateDT,year
0,1965-01-29,A,0,1965-01-29,1965
1,1965-01-29,AA,1,1965-01-29,1965
2,1965-01-29,AAP,0,1965-01-29,1965
3,1965-01-29,ABC,0,1965-01-29,1965
4,1965-01-29,ABP,1,1965-01-29,1965


In [33]:
data_raw.head()

,PERMNO,date,SHRCD,TICKER,COMNAM,PRC,VOL,RET,RETX,dateDT,year,id
0,10495,1965-01-29,10.0,A,ANACONDA CO,60.500,4337.0,0.157895,0.157895,1965-01-29,1965,0
1,24643,1965-01-29,11.0,AA,ALUMINUM COMPANY AMER,62.500,1849.0,0.016260,0.016260,1965-01-29,1965,1
2,28863,1965-01-29,10.0,AAP,ALLIED ARTISTS PICTURES CORP,2.125,436.0,0.133333,0.133333,1965-01-29,1965,2
3,25101,1965-01-29,10.0,ABC,A B C CONS CORP,19.750,930.0,0.144928,0.144928,1965-01-29,1965,3
4,10233,1965-01-29,10.0,ABP,AMERICAN BROADCASTING PARAMOUNT,52.500,1390.0,-0.016393,-0.016393,1965-01-29,1965,4


In [6]:
if not isinstance(data_raw['date'][0], str):
    data_raw["date"] = data_raw["date"].apply(lambda x: str(x)[:4]+'-'+str(x)[4:6]+'-'+str(x)[6:])
data_raw["dateDT"] = pd.to_datetime(data_raw["date"])
data_raw["year"] = [_.year for _ in data_raw["dateDT"]]

if not isinstance(y_raw['date'][0], str):
    y_raw["date"] = y_raw["date"].apply(lambda x: str(x)[:4]+'-'+str(x)[4:6]+'-'+str(x)[6:])
y_raw["dateDT"] = pd.to_datetime(y_raw["date"])
y_raw["year"] = [_.year for _ in y_raw["dateDT"]]

In [7]:
data_raw = data_raw.sort_values(by=["dateDT", "TICKER"])
data_raw.index = range(data_raw.shape[0])

In [8]:
y_raw = y_raw.sort_values(by=["dateDT", "TICKER"])
y_raw.index = range(y_raw.shape[0])

In [32]:
data_raw["id"] = data_raw.index

In [34]:
# split train, val, test
train_val_split = int(data_raw.shape[0]*0.4)
val_test_split = int(data_raw.shape[0]*0.6)
data_train = data_raw.loc[data_raw["dateDT"] <= data_raw["dateDT"][train_val_split]]
train_cut = data_train.shape[0]
data_val = data_raw.iloc[train_cut:].loc[data_raw["dateDT"] <= data_raw["dateDT"][val_test_split]]
val_cut = train_cut + data_val.shape[0]
data_test = data_raw.iloc[val_cut:]

In [35]:
y_train = y_raw.iloc[:train_cut]
y_val = y_raw.iloc[train_cut:val_cut]
y_test = y_raw.iloc[val_cut:]

# Descriptive Analysis

# Basic Models